# OLX case study: Final models

This last notebook demonstrates how to import and deploy the created models. They can be used to assess the blurriness of images and to classify whether or not they show a clock. As a summary, the predictions for all images provided by OLX are collected in a JSON file.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import os
import pathlib
import json

import requests
from PIL import Image

from models import OLXModel

In [ ]:
data_dir = '../data' # data directory
weights_file = '../weights.pt' # stored weights

In [ ]:
model = OLXModel(weights_file) # predicts blurriness and clockness

In [ ]:
summary = {}
for subdir, dirs, files in os.walk(data_dir):
    file_names = sorted([f for f in files if f.endswith(('.jpg', '.jpeg'))])

    for file_name in file_names:
        file_path = os.path.join(subdir, file_name)
        image = Image.open(file_path)

        blurriness, clockness, is_clock = model(
            image,
            transform_mode='full',
            threshold=0.5
        )

        summary[file_name] = {
            'blurriness': blurriness,
            'clockness': clockness,
            'isclock': is_clock
        }

In [ ]:
with open('../summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

Assuming that a Flask inference server is already running, we can query model predictions also as shown below. Note that both example images are correctly classified. Though, it is remarked that they might have been part of the training set.

In [ ]:
files = ('551_2.jpg', '3255325488.jpg')

predictions = []
for file_name in files:
    with open(pathlib.Path(data_dir) / file_name, 'rb') as f:
        image_bytes = f.read()
    r = requests.post(url='http://localhost:5000/predict', files={'image': image_bytes}).json()
    predictions.append(r)

print('Other image:', predictions[0])
print('Clock image:', predictions[1])